In [1]:
from huggingface_hub import notebook_login
import pandas as pd

In [3]:
#notebook_login()

In [8]:
#doesn't seem to run  for some reason

from datasets import load_dataset
reddit = load_dataset('reddit')

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# Output saved
Selection deleted
#doesn't seem to run  for some reason

from datasets import load_dataset
reddit = load_dataset('reddit')

Downloading and preparing dataset reddit/default to C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e...

Downloading data:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e. Subsequent calls will reuse this data.

  0%|          | 0/1 [00:00<?, ?it/s]

# Subset the data
Want a data set ~ 10k in size 
Will choose the first 10k league of legends subreddit entries


In [53]:
#just loop thorugh taking only the league of legends subreddit data
N = 10000
S_N = N/10
I = 0
J = 0
subreddit = 'leagueoflegends'
data_store = []

while(J < N):
    data = reddit['train'][I]
    data_bool = data['subreddit']
    if data_bool == subreddit:
        #print(J)
        J += 1
        data_store.append(data)

        if (J % S_N) ==0:
            print(J) 

    I += 1

        
    
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [54]:
df = pd.DataFrame(data_store)
df.to_csv(f'data/reddit_{subreddit}_N_{N}.csv')

# Now have a data set
Only need the body and summary columns

In [68]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

In [58]:
name = 'data/reddit_leagueoflegends_N_10000.csv'
df = pd.read_csv(name)

In [69]:
# convert into data_loader object


reddit_lol = load_dataset("csv", data_files=name)

Found cached dataset csv (C:/Users/AT030915/.cache/huggingface/datasets/csv/default-7267e4e3846f4574/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\AT030915\HuggingFace_Text_Summarisation\venv\HF_venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AT030915\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [72]:
prefix = "summarize: " #choose prefix so t5 summarises

#define function to 
def preprocess_function(examples):
    main = 'body' #main field name
    summary = 'summary' #summary field name 
    
    inputs = [prefix + doc for doc in examples[main]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples[summary], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [73]:
tokenized_reddit_lol = reddit_lol.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]